In [2]:
!pip install pyspark

The second dataset and find which set of features you have to use to better predict the performance index of the students.

In [3]:
from pyspark.sql import SparkSession

Upload database on the CSV file or other compatible dataset file to Colab.Create a SparkSession

In [10]:
spark = SparkSession.builder.appName("StudentPerformancePrediction").getOrCreate()

Read data

In [11]:
data = spark.read.csv("Student_Performance.csv", header=True, inferSchema=True)

Show database for the First Few Rows



In [12]:
data.show(5)

+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+
|Hours Studied|Previous Scores|Extracurricular Activities|Sleep Hours|Sample Question Papers Practiced|Performance Index|
+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+
|            7|             99|                       Yes|          9|                               1|             91.0|
|            4|             82|                        No|          4|                               2|             65.0|
|            8|             51|                       Yes|          7|                               2|             45.0|
|            5|             52|                       Yes|          5|                               2|             36.0|
|            7|             75|                        No|          8|                               5|             66.0|
+-------------+---------

In [ ]:
!pip install findspark

Imort labreries from pyspark

In [5]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder,StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

Vectorize Features - Combine the selected features into a single vector using

In [13]:
assembler = VectorAssembler(
    inputCols=["Hours Studied", "Previous Scores","Sleep Hours","Sample Question Papers Practiced","Performance Index"],
                            outputCol="features")

Data Scalin,
Choose a Regression Model and
Apply Feature Engineering pipeline

In [15]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
learnReg = LinearRegression(featuresCol="scaledFeatures", labelCol="Performance Index")
pipeline = Pipeline(stages=[assembler, scaler, learnReg])

Split data

randomSplit([0.8 , 0.2])
Training set: 80 data points (80% of the original data)
Testing set: 20 data points (20% of the original data)

In [17]:
train_data , test_data = data.randomSplit([0.8 ,0.2] , seed=42)

Train the model

In [18]:
model =pipeline.fit(train_data)

Make Prediction

In [19]:
predictions = model.transform(train_data)

In [20]:
predictions.show(2)

+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+--------------------+--------------------+------------------+
|Hours Studied|Previous Scores|Extracurricular Activities|Sleep Hours|Sample Question Papers Practiced|Performance Index|            features|      scaledFeatures|        prediction|
+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+--------------------+--------------------+------------------+
|            1|             40|                        No|          4|                               3|             15.0|[1.0,40.0,4.0,3.0...|[0.38500154530584...|15.000000000091521|
|            1|             40|                        No|          4|                               8|             12.0|[1.0,40.0,4.0,8.0...|[0.38500154530584...|12.000000000016977|
+-------------+---------------+--------------------------+-----------+---------------

Import Libreris

In [21]:
from pyspark.ml.evaluation import RegressionEvaluator

Create an Evolutor

In [22]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Performance Index", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 3.94395797985798e-11


Calculate Performance

In [ ]:
from pyspark.ml.tuning import CrossValidator

Additional Metrics

In [24]:
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 3.1461827518222426e-11


Show Predictions for the First Few Rows

In [23]:
predictions.select("features", "Performance Index", "prediction").show(5)

+--------------------+-----------------+------------------+
|            features|Performance Index|        prediction|
+--------------------+-----------------+------------------+
|[1.0,40.0,4.0,3.0...|             15.0|15.000000000091521|
|[1.0,40.0,4.0,8.0...|             12.0|12.000000000016977|
|[1.0,40.0,5.0,9.0...|             10.0| 9.999999999964098|
|[1.0,40.0,5.0,9.0...|             14.0|14.000000000033365|
|[1.0,40.0,6.0,0.0...|             15.0|15.000000000077621|
+--------------------+-----------------+------------------+
only showing top 5 rows



R-squared: close to 1 indicates that the model explains the variance of the data well.